# Kotani & Hughes 2017 -- Autoatalytic system

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [ ]:
def sema(vect, fig):
    """Rewrite the semantics column into a more compact, human readable version.
    """
    nv = []
    for line in vect:
        v = []
        for (x,y) in fig.pepperargs[line].items():
            if x == 'enumconc': continue
            if x == 'max_complex_size': continue
            if x == 'max_complex_count': continue
            if x == 'max_reaction_count': continue
            if x == 'condensed': 
                z = x if y is True else 'detailed'
            else:
                z = '{}={}'.format(x,y)
            v.append(z)
        nv.append(', '.join(v))
    return nv

In [ ]:
from kotani2017 import data; k17 = data()

k17 = [x for x in k17 if 'Fig. 4' in x.name] # Filter to use only Figure 4.
assert len(k17) == 1
[F4] = k17

### Time detailed vs condensed runs (that takes a while)

In [ ]:
%%timeit
verb = 0
for fig in k17:
    assert fig.name == 'Kotani & Hughes (2017) Fig. 4 - Autocatalytic DSD system with 4-way branch migration (varying autocatalyst)'
    fig.pepperargs['#1-detailed']  = {'condensed': False, 'enumconc': 'nM', 'release_cutoff': 8, 'max_complex_size': 10}
    fig.pepperargs['#3-detailed']  = {'condensed': False, 'enumconc': 'nM', 'k_slow': 1e-4, 'max_complex_size': 10}
    fig.pepperargs['#4-detailed']  = {'condensed': False, 'enumconc': 'nM', 'k_slow': 1e-4, 'k_fast': 1e-3, 'max_complex_size': 16, 'max_complex_count': 10000, 'max_reaction_count': 10000}
    fig.pepperargs['#5-detailed']  = {'condensed': False, 'enumconc': 'nM', 'k_slow': 1e-4, 'k_fast': 1e-2, 'max_complex_size': 24, 'max_complex_count': 10000, 'max_reaction_count': 10000} # solver cannot handle leak
    fig.pepperargs['#6-detailed']  = {'condensed': False, 'enumconc': 'nM', 'k_slow': 1e-5, 'k_fast': 1e-2, 'max_complex_size': 24, 'max_complex_count': 10000, 'max_reaction_count': 10000}
    
    fig.eval('#1-detailed',  verbose = verb, enumprofile = True)
    fig.eval('#3-detailed',  verbose = verb, enumprofile = True)
    fig.eval('#4-detailed',  verbose = verb, enumprofile = True)
    fig.eval('#5-detailed',  verbose = verb, enumprofile = True)
    fig.eval('#6-detailed',  verbose = verb, enumprofile = True)

In [ ]:
%%timeit
verb = 0
for fig in k17:
    assert fig.name == 'Kotani & Hughes (2017) Fig. 4 - Autocatalytic DSD system with 4-way branch migration (varying autocatalyst)'
    fig.pepperargs['#1-condensed'] = {'condensed': True,  'enumconc': 'nM', 'release_cutoff': 8, 'max_complex_size': 10}  
    fig.pepperargs['#3-condensed'] = {'condensed': True,  'enumconc': 'nM', 'k_slow': 1e-4, 'max_complex_size': 10}
    fig.pepperargs['#4-condensed'] = {'condensed': True,  'enumconc': 'nM', 'k_slow': 1e-4, 'k_fast': 1e-3, 'max_complex_size': 16, 'max_complex_count': 10000, 'max_reaction_count': 10000}
    fig.pepperargs['#5-condensed'] = {'condensed': True,  'enumconc': 'nM', 'k_slow': 1e-4, 'k_fast': 1e-2, 'max_complex_size': 24, 'max_complex_count': 10000, 'max_reaction_count': 10000} # solver cannot handle leak
    fig.pepperargs['#6-condensed'] = {'condensed': True,  'enumconc': 'nM', 'k_slow': 1e-5, 'k_fast': 1e-2, 'max_complex_size': 24, 'max_complex_count': 10000, 'max_reaction_count': 10000}
    
    fig.eval('#1-condensed', verbose = verb, enumprofile = True)
    fig.eval('#3-condensed', verbose = verb, enumprofile = True)
    fig.eval('#4-condensed', verbose = verb, enumprofile = True)
    fig.eval('#5-condensed', verbose = verb, enumprofile = True)
    fig.eval('#6-condensed', verbose = verb, enumprofile = True) 

### Do analysis for plots ...

In [ ]:
def load_old_results(fig, basename, cmpfig):
    fig._enumerated |= set([basename + '-enum.pil'])
    fig._simulated |= set([basename + '-C1=0.1-simu',
                            basename + '-C1=0.01-simu',
                            basename + '-C1=0.001-simu',
                            basename + '-C1=0-simu'])
    fig._simexecs |= set([basename + '-simu.py'])
    F4.cmpfig[cmpfig] = basename + '-cmp.nxy'

if False: # load old results instead of calculating them again?
    load_old_results(F4, "tmp/Kotani2017-F4-00-#1-detailed", "#1-detailed")
    load_old_results(F4, "tmp/Kotani2017-F4-00-#1-condensed", "#1-condensed")
    load_old_results(F4, "tmp/Kotani2017-F4-00-#3-detailed", "#3-detailed")
    load_old_results(F4, "tmp/Kotani2017-F4-00-#3-condensed", "#3-condensed")
    load_old_results(F4, "tmp/Kotani2017-F4-00-#4-detailed", "#4-detailed")
    load_old_results(F4, "tmp/Kotani2017-F4-00-#4-condensed", "#4-condensed")
    load_old_results(F4, "tmp/Kotani2017-F4-00-#5-detailed", "#5-detailed")
    load_old_results(F4, "tmp/Kotani2017-F4-00-#5-condensed", "#5-condensed")
    load_old_results(F4, "tmp/Kotani2017-F4-00-#6-detailed", "#6-detailed")
    load_old_results(F4, "tmp/Kotani2017-F4-00-#6-condensed", "#6-condensed")

In [ ]:
Table1 = pd.DataFrame()

verb = 1
for fig in k17:
    assert fig.name == 'Kotani & Hughes (2017) Fig. 4 - Autocatalytic DSD system with 4-way branch migration (varying autocatalyst)'
    fig.pepperargs['#1-detailed']  = {'condensed': False, 'enumconc': 'nM', 'release_cutoff': 8, 'max_complex_size': 10}
    fig.pepperargs['#1-condensed'] = {'condensed': True,  'enumconc': 'nM', 'release_cutoff': 8, 'max_complex_size': 10}
    fig.pepperargs['#2-detailed']  = {'condensed': False, 'enumconc': 'nM', 'k_slow': 1e-3, 'max_complex_size': 20} # doesn't work, no 4-way branch migration, no D
    fig.pepperargs['#2-condensed'] = {'condensed': True,  'enumconc': 'nM', 'k_slow': 1e-3, 'max_complex_size': 20} # doesn't work, no 4-way branch migration, no D
    
    fig.pepperargs['#3-detailed']  = {'condensed': False, 'enumconc': 'nM', 'k_slow': 1e-4, 'max_complex_size': 10}
    fig.pepperargs['#3-condensed'] = {'condensed': True,  'enumconc': 'nM', 'k_slow': 1e-4, 'max_complex_size': 10}
    fig.pepperargs['#4-detailed']  = {'condensed': False, 'enumconc': 'nM', 'k_slow': 1e-4, 'k_fast': 1e-3, 'max_complex_size': 16, 'max_complex_count': 10000, 'max_reaction_count': 10000}
    fig.pepperargs['#4-condensed'] = {'condensed': True,  'enumconc': 'nM', 'k_slow': 1e-4, 'k_fast': 1e-3, 'max_complex_size': 16, 'max_complex_count': 10000, 'max_reaction_count': 10000}
    fig.pepperargs['#5-detailed']  = {'condensed': False, 'enumconc': 'nM', 'k_slow': 1e-4, 'k_fast': 1e-2, 'max_complex_size': 24, 'max_complex_count': 10000, 'max_reaction_count': 10000} # solver cannot handle leak
    fig.pepperargs['#5-condensed'] = {'condensed': True,  'enumconc': 'nM', 'k_slow': 1e-4, 'k_fast': 1e-2, 'max_complex_size': 24, 'max_complex_count': 10000, 'max_reaction_count': 10000} # solver cannot handle leak
    fig.pepperargs['#6-detailed']  = {'condensed': False, 'enumconc': 'nM', 'k_slow': 1e-5, 'k_fast': 1e-2, 'max_complex_size': 24, 'max_complex_count': 10000, 'max_reaction_count': 10000}
    fig.pepperargs['#6-condensed'] = {'condensed': True,  'enumconc': 'nM', 'k_slow': 1e-5, 'k_fast': 1e-2, 'max_complex_size': 24, 'max_complex_count': 10000, 'max_reaction_count': 10000}
    
    fig.eval('#1-detailed',  verbose = verb, cmpfig = True)
    fig.eval('#1-condensed', verbose = verb, cmpfig = True)
    fig.eval('#3-detailed',  verbose = verb, cmpfig = True)
    fig.eval('#3-condensed', verbose = verb, cmpfig = True)
    fig.eval('#4-detailed',  verbose = verb, cmpfig = True)
    fig.eval('#4-condensed', verbose = verb, cmpfig = True)
    fig.eval('#5-detailed',  verbose = verb, cmpfig = True)
    fig.eval('#5-condensed', verbose = verb, cmpfig = True)
    fig.eval('#6-detailed',  verbose = verb, cmpfig = True)
    fig.eval('#6-condensed', verbose = verb, cmpfig = True) 

    for df in fig.get_system_dataframes():
        df['Name'] = fig.name
        df['condensed'] = 'condensed' if any(map(lambda x: 'condensed' in x, df['Semantics'])) else 'detailed'
        df['pargs'] = list(map(lambda x: x[:2], df['Semantics']))
        df['Semantics']=sema(df['Semantics'], fig)
        Table1 = Table1.append(df)


# Plotting Data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

In [ ]:
tmpfig = Table1.copy()
tmpfig = tmpfig.loc[tmpfig['Name'].isin(['Kotani & Hughes (2017) Fig. 4 - Autocatalytic DSD system with 4-way branch migration (varying autocatalyst)'])]
tmpfig = tmpfig.loc[tmpfig['Metric'].isin(['completion-time'])]
tmpfig.to_latex('Kotani2017data.tex', index = False, float_format = '{:.0f}'.format)
#display(tmpfig)

with sns.plotting_context('paper', font_scale = 1.5):
    g = sns.lmplot(x="Time (experiment)", 
               y="Time (simulation)", 
               hue="pargs", col='condensed', data=tmpfig, scatter_kws={"s": 50, "alpha": 1},
               fit_reg=False, hue_order=['#1', '#2', '#3', '#4', '#5', '#6'], markers=['v', 'x', '^', '.','x','+'])
    
g.set_titles(row_template = '{row_name}', col_template = '{col_name}')
plt.gcf().set_size_inches(4.5,3.2)

plt.xscale('log')
plt.yscale('log')
(mi, ma)=(1e3, 1e5)
plt.xlim(0.9e3, 1.1e5)
plt.ylim(0.9e2, 1.1e5)
for ax in plt.gcf().get_axes():
    ax.plot([mi, ma], [mi, ma], linewidth=1, color='white',zorder=0)
    ax.set(xlabel='$t_{1/2}$ experiment [s]')
plt.gca().set(ylabel='$t_{1/2}$ simulation [s]')

plt.savefig('kotani2017_F4_params.pdf', bbox_inches='tight')
plt.savefig('kotani2017_F4_params.svg', bbox_inches='tight')

In [ ]:
# Compare reporter trajectory for different initial conditions.

# Get Data from File
[F4] = k17

nxy = np.loadtxt(F4.cmpfig['#1-detailed'], skiprows = 1)
legend = ['$C_0 = 100$ pM', '$C_0 = 10$ pM', '$C_0 = 1$ pM', '$C_0 = 0$ pM']
time = nxy[:,0] / 3600
ys = nxy[:,1:]

plt.gca().set_prop_cycle(color=['orange','blue', 'red', 'black'])
#plt.figure(figsize=(3.5,2.5))
plt.plot(time, ys)

plt.xlim(-0.5,15.5)
plt.xticks(np.arange(0, 16, step=5), fontsize=10)
plt.ylim(-0.5,10.5)
plt.yticks(np.arange(0, 11, step=2.5), fontsize=10)

plt.legend(legend, ncol=1, fontsize=8);
plt.gca().set_xlabel('Time [hours]', fontsize=10)
plt.gca().set_ylabel('Concentration D [nM]', fontsize=10)
plt.gca().axhline(y=5, linewidth=1, color='black', linestyle='--')

#plt.savefig('sim_kotani17.pdf', bbox_inches='tight')
#plt.savefig('sim_kotani17.svg', bbox_inches='tight')